In [131]:
import spacy
import pandas as pd

from spacy.tokenizer import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup

from textblob import TextBlob

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore


In [145]:
nlp = spacy.load("en_core_web_lg")

In [98]:
# Sams emails
emails = pd.read_excel('Sam-emails.xlsx')

In [59]:
def clean_categories(text):
    text = text.replace("Promotions", '')
    text = text.replace("'", '')
    text = text.strip("[").strip("]")
    text = text.strip("'").strip(",").strip(" ")
    text = text.split(",")
    return text

emails['tags'] = emails['tags'].apply(clean_categories)
emails.head()

,keys,from/to,subject,body,tags
0,809,Google <no-reply@accounts.google.com>,App password created,--00000000000002379205972971fd Content-Type: t...,[Security]
1,808,Google <no-reply@accounts.google.com>,2-Step Verification turned on,--000000000000d50ed3059729649c Content-Type: t...,[Security]
2,807,"""Medium Daily Digest"" <noreply@medium.com>",How To Wake Up at 5 A.M. Every Day | Bryan Ye ...,--337b1df667177cb513899ad44e31fb8d4f8637575996...,[Productivity]
3,806,"""no-reply-aws@amazon.com"" <no-reply-aws@amazon...",RE:[CASE 6570793521] Limit Increase: SageMaker,------=_Part_26869745_214056034.1573352866590 ...,[Productivity]
4,805,"""Medium Daily Digest"" <noreply@medium.com>",Multiclass Text Classification From Start To F...,--c4169d46c40775d3344614c9be2f1235565dfa74c04d...,[Productivity]


In [99]:
# Checking emails for text content type
# Only 42 were not labeled as that content type

# mask = (emails['body'].str.contains("text/plain") | emails['body'].str.contains("text/html"))
# print('emails.shape', emails.shape)
# emails_text = emails[mask]
# print('emails_text.shape', emails_text.shape)

emails.shape (808, 5)
emails_text.shape (766, 5)


In [71]:
# Emails without content are still clean text in Sam's emails. Go without this method for now

# emails_not_text = emails[~mask]
# emails_not_text

,keys,from/to,subject,body,tags
10,799,"""Office of the Registrar"" <registrar@uw.edu>",Transcript Order payment confirmation,This is to confirm that a credit card payment ...,[Finance]
12,797,Amazon Web Services <no-reply-aws@amazon.com>,Amazon Web Services: You have opened a new Sup...,Thank you for contacting Amazon Web Services. ...,[Productivity]
13,796,Amazon Web Services <no-reply-aws@amazon.com>,[Case 6570793521] Limit Increase: SageMaker,Limit increase request 1 Service: SageMaker Re...,[Productivity]
30,779,GitHub <noreply@github.com>,[GitHub] A third-party OAuth application has b...,Hey SamH3pn3r! A third-party OAuth applicatio...,"[Productivity, other]"
40,769,"""Leif"" <support@leif.org>",NaN,NaN,[Finance]
41,768,"""Leif"" <support@leif.org>",NaN,NaN,[Finance]
44,765,notifications@typeform.com,Austen Allred: Thank you for filling out DS Fu...,"Hello, and welcome to Lambda School.<br /><br ...",[Productivity]
58,751,turbotax@intuit.com,TurboTax Update: Federal Return Accepted,<!doctype html> <html> <head> <title></t...,[Finance]
65,744,eSCRIP-SAFE <escrip-safe@escrip-safe.com>,Notice of transcript receipt,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 S...",[Finance]
66,743,eSCRIP-SAFE <escrip-safe@escrip-safe.com>,Your transcript has been transmitted,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 S...",[Security]


In [86]:
# Functions for checking NLP measures of sentiment

def polarity_txt(text):
    return TextBlob(text).sentiment[0]

def subj_txt(text):
    return  TextBlob(text).sentiment[1]

def len_txt(text):
    if len(text.split())>0:
         return len(set(clean_txt(text).split()))/ len(text.split())
    else:
         return 0

In [100]:
# Implement the sentiment features

emails['body'] = emails['body'].apply(str)

emails['polarity'] = emails['body'].apply(polarity_txt)

emails['subjectivity'] = emails['body'].apply(subj_txt)

# emails['len'] = emails['body'].apply(len_txt)

emails.head()

,keys,from/to,subject,body,tags,polarity,subjectivity
0,809,Google <no-reply@accounts.google.com>,App password created,--00000000000002379205972971fd Content-Type: t...,['Security'],0.293939,0.533333
1,808,Google <no-reply@accounts.google.com>,2-Step Verification turned on,--000000000000d50ed3059729649c Content-Type: t...,Security'],0.252778,0.456713
2,807,"""Medium Daily Digest"" <noreply@medium.com>",How To Wake Up at 5 A.M. Every Day | Bryan Ye ...,--337b1df667177cb513899ad44e31fb8d4f8637575996...,"['Promotions', 'Productivity']",0.292987,0.590647
3,806,"""no-reply-aws@amazon.com"" <no-reply-aws@amazon...",RE:[CASE 6570793521] Limit Increase: SageMaker,------=_Part_26869745_214056034.1573352866590 ...,['Productivity'],0.140374,0.416444
4,805,"""Medium Daily Digest"" <noreply@medium.com>",Multiclass Text Classification From Start To F...,--c4169d46c40775d3344614c9be2f1235565dfa74c04d...,"['Promotions', 'Productivity']",0.283817,0.497817


In [95]:
emails.body[0].split()

['--00000000000002379205972971fd',
 'Content-Type:',
 'text/plain;',
 'charset="UTF-8";',
 'format=flowed;',
 'delsp=yes',
 'Content-Transfer-Encoding:',
 'base64',
 'ImltYXAiIHBhc3N3b3JkIHN1Y2Nlc3NmdWxseSBjcmVhdGVkDQoNCg0KDQpIaSBTYW11ZWwsDQpZ',
 'b3UgaGF2ZSBzdWNjZXNzZnVsbHkgY3JlYXRlZCBhbiBhcHAgcGFzc3dvcmQgZm9yIHlvdXIgR29v',
 'Z2xlIEFjY291bnQNCnNhbXVlbGhlcG5lcjAzQGdtYWlsLmNvbSBuYW1lZCBpbWFwIG9uIFR1ZXNk',
 'YXksIE5vdmVtYmVyIDEyLCAyMDE5IDk6MTggQU0NCihQVCkuDQoNClRoaXMgYXBwIHBhc3N3b3Jk',
 'IHdpbGwgYWxsb3cgeW91IHRvIGFjY2VzcyB5b3VyIEdvb2dsZSBhY2NvdW50IGZyb20gYQ0KZGV2',
 'aWNlIG9yIGFwcGxpY2F0aW9uIHRoYXQgY2FuIG9ubHkgYmUgY29uZmlndXJlZCB3aXRoIGEgdXNl',
 'cm5hbWUgYW5kDQpwYXNzd29yZCwgcmF0aGVyIHRoYW4gYSB1c2VybmFtZSwgcGFzc3dvcmQgYW5k',
 'IGEgdmVyaWZpY2F0aW9uIGNvZGUuDQoNCipEb24ndCByZWNvZ25pemUgdGhpcyBhY3Rpdml0eT8q',
 'DQpSZXZpZXcgeW91ciByZWNlbnRseSB1c2VkIGRldmljZXMNCjxodHRwczovL2FjY291bnRzLmdv',
 'b2dsZS5jb20vQWNjb3VudENob29zZXI/RW1haWw9c2FtdWVsaGVwbmVyMDNAZ21haWwuY29tJmNv',
 'bnRpbnVlPWh0dHBzOi8vbXl

In [154]:
# Tokenize functions currently

STOPWORDS = set(STOPWORDS).union(set(['a','Content-Type:', 'tf', 'td', 'size', 'font', 'content', 'type', 'text', 'plain', 'charset', ]))

# Gensim preprocess
def tokenize(text):
    return [token for token in simple_preprocess(text) if (token not in STOPWORDS and len(token) >= 3)]

# # Spacy Tokenizer
# def tokenize(document):
#     doc = nlp(document)
#     return [token.lemma_ for token in doc if (token not in STOPWORDS) and (token.is_punct != True) and (len(token) >= 3)]

In [157]:
emails['tokens'] = emails['body'].apply(tokenize)
emails['tokens']

0      [utf, format, flowed, delsp, yes, transfer, en...
1      [utf, format, flowed, delsp, yes, transfer, en...
2      [fbe, transfer, encoding, quoted, printable, u...
3      [utf, transfer, encoding, bit, hello, hope, ha...
4      [dfa, transfer, encoding, quoted, printable, u...
5      [utf, transfer, encoding, quoted, printable, b...
6      [utf, transfer, encoding, quoted, printable, t...
7      [bcf, eaf, dfd, transfer, encoding, quoted, pr...
8      [eff, ebbf, transfer, encoding, quoted, printa...
9      [utf, transfer, encoding, bit, aws, free, tier...
10     [confirm, credit, card, payment, website, toda...
11     [utf, transfer, encoding, bit, hope, day, trea...
12     [thank, contacting, amazon, web, services, ope...
13     [limit, increase, request, service, sagemaker,...
14     [multipart, mixed, boundary, multipart, altern...
15     [ffb, daaa, transfer, encoding, quoted, printa...
16     [cbajefw, dwlgqmnqce, lua, utf, transfer, enco...
17     [utf, transfer, encoding

In [164]:
emails['body'][1]

'--000000000000d50ed3059729649c Content-Type: text/plain; charset="UTF-8"; format=flowed; delsp=yes Content-Transfer-Encoding: base64  Mi1TdGVwIFZlcmlmaWNhdGlvbiB0dXJuZWQgb24NCg0KDQoNCkhpIFNhbXVlbCwNCkNvbmdyYXR1 bGF0aW9ucyBvbiBlbmFibGluZyAyLVN0ZXAgVmVyaWZpY2F0aW9uIGZvciB5b3VyIEdvb2dsZSBB Y2NvdW50DQpzYW11ZWxoZXBuZXIwM0BnbWFpbC5jb20hIFdlIGhhdmUgYSBjb3VwbGUgb2Ygc3Vn Z2VzdGlvbnMgdG8gbWFrZSBzdXJlIHlvdQ0KY2FuIGFsd2F5cyBhY2Nlc3MgeW91ciBhY2NvdW50 Lg0KDQoqU2V0IHVwIGEgYmFja3VwIHBob25lKg0KSWYgeW91ciBwcmltYXJ5IHBob25lIGlzIGV2 ZXIgdW5hdmFpbGFibGUgd2hlbiB5b3UgbmVlZCBhIGNvZGUsIEdvb2dsZSBjYW4NCnNlbmQgb25l IHRvIGEgYmFja3VwLCBsaWtlIHlvdXIgaG9tZSBvciB3b3JrIHBob25lIG9yIGV2ZW4gYSBmYW1p bHkNCm1lbWJlcidzIHBob25lLiBUbyBzZXQgb25lIHVwLA0KDQogICAgMS4gR28gdG8geW91ciAy LVN0ZXAgVmVyaWZpY2F0aW9uIHNldHRpbmdzDQogICAgIA0KPGh0dHBzOi8vYWNjb3VudHMuZ29v Z2xlLmNvbS9BY2NvdW50Q2hvb3Nlcj9FbWFpbD1zYW11ZWxoZXBuZXIwM0BnbWFpbC5jb20mY29u dGludWU9aHR0cHM6Ly9teWFjY291bnQuZ29vZ2xlLmNvbS9zaWduaW5vcHRpb25zL3R3by1zdGVw LXZlcmlmaWNhdGlvbj

In [155]:
tfidf = TfidfVectorizer(tokenizer=tokenize, analyzer='word', strip_accents='unicode', min_df=0.025, max_df=.95, ngram_range=(1,2))

sparse = tfidf.fit_transform(emails['body'])

In [156]:
dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

dtm.head()

,aaa,aaaa,aaad,aab,aad,aaecaxeebsex,aaecaxeebsex rcygromjygpkju,aaf,aaf fhbyjxfdkbkaeii,aap,...,zzs,zzs xojig,zzs xojignca,zzxjp,zzxjpzii,zzxjpzjsncgljb,zzxjpzjsncgljb xvcjojmuy,zzxjpzjt,zzxjpzjtjb,zzxjpzjtjb xvcjojmuy
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.016173,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
